## Main paths

In [3]:
# Folder where demos are stored
path_to_demos = "..\\input"
# Folder to output dataframe
output = "..\\output"
# Folder with txt file for demos to view
path_to_txt = "..\\Demos_a_revisar.txt"

# Extracting zips

In [ ]:
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib

os.chdir(path_to_demos)

# List of rounds to check manually
timeOuts = []
# Dataframe with output
output = pd.DataFrame()

carpetas = []
for file in os.listdir():
    if ".rar" in file:
        patoolib.extract_archive("%s" % file, outdir="%s" %path_to_demos)

    # Agarro los nombres de las demos
    demos = []
    for demo in os.listdir():
        if (".dem" in demo) and (demo[0]!="x"):
            os.rename(demo, "x" + file.split("vs")[0] + "vs" + demo.split("vs")[1])
            demos += ["x" + file.split("vs")[0] + "vs" + demo.split("vs")[1]]
        if ".json" in demo:
            os.remove(demo)
            
    ### Itero el parser por cada demo
    for demo in demos:
        demo_parser = DemoParser(
        demofile = "%s" % demo,
        parse_rate=128, 
        buy_style="hltv"
        )
    
    
        # Parse the demofile, output results to dictionary
        df = demo_parser.parse(return_type="df")
    
        # Agrego una columna que mide los ticks entre el comienzo de la ronda y final del freezeTime
        df["rounds"]["freezeTimeTotal"] = df["rounds"]["freezeTimeEndTick"] - df["rounds"]["startTick"]
        
        # Adjunto el ID de la partida y la ronda a revisar manualmente a la lista timeOuts
        for index, row in df["rounds"].iterrows():
            if row["freezeTimeTotal"] != 2560 and row["roundNum"] != 1:
                timeOuts += ["Ver %s en ronda %s" % (row["matchID"], row["roundNum"])]
        
        output = pd.concat([output, df["rounds"]], ignore_index = True)
        
        os.remove(demo)
        
output.drop(output.columns[17:25], axis = 1, inplace = True)

with pd.ExcelWriter(
    f"{output}\\parsedDemos.xlsx",  # path donde dejar las demos parseadas
    mode = "a",
    if_sheet_exists = "overlay"
) as writer:
    output.to_excel(writer, index = False, startrow = 1, header = None)

with open('.\\Demos_a_revisar.txt', 'a') as f:
    f.writelines("\n".join(timeOuts))

## Without extracting zips

In [ ]:
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib

# Demo list to process
os.chdir(path_to_demos)

# Grab demo names
demos = []
for file in os.listdir():
    if ".dem" in file:
        demos += [file]
    
# list of rounds to check manually
timeOuts = []
# output dataframe
output = pd.DataFrame()

### Iterate parser over demos
for demo in demos:
    demo_parser = DemoParser(
    demofile = "%s" % demo,
    parse_rate=128, 
    buy_style="hltv"
    )


    # Parse the demofile, output results to dictionary
    df = demo_parser.parse(return_type="df")

    # Add column measing ticks between start of the round and end of freeze time
    df["rounds"]["freezeTimeTotal"] = df["rounds"]["freezeTimeEndTick"] - df["rounds"]["startTick"]
    
    # Take match ID and round number and add it to timeout list
    for index, row in df["rounds"].iterrows():
        if row["freezeTimeTotal"] != 2560 and row["roundNum"] != 1:
            timeOuts += ["Ver %s en ronda %s" % (row["matchID"], row["roundNum"])]
    
    output = pd.concat([output, df["rounds"]], ignore_index = True)

# Drop unnecesary variables (these are potential outcomes)
output.drop(output.columns[17:25], axis = 1, inplace = True)

with pd.ExcelWriter(
    f"{output}\\parsedDemos.xlsx",  # path donde dejar las demos parseadas
    mode = "a",
    if_sheet_exists = "overlay"
) as writer:
    output.to_excel(writer, index = False, startrow = 1, , header = None)

with open('.\\Demos_a_revisar.txt', 'a') as f:
    f.writelines("\n".join(timeOuts))